In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd

X = pd.read_csv('First76.csv').drop(['Class', 'Sequence'], axis=1)
y = pd.read_csv('First76.csv')['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [11]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2181, 18) (935, 18) (2181,) (935,)


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load the data
df = pd.read_csv('First76.csv')  # Adjust `sep` if necessary

# One-hot encode the 'Class' column
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(df[['Class']])  # Convert 'Class' into one-hot encoding

# Prepare X (features) by dropping 'Class' and 'Sequence' columns
X = df.drop(['Class', 'Sequence'], axis=1)

# Reshape X to 3D format (samples, timesteps, features)
# Group by 'Sequence' to maintain sequences
X_sequences = []
y_sequences = []

for _, group in df.groupby(['Class', 'Sequence']):
    X_seq = group.drop(['Class', 'Sequence'], axis=1).values  # Drop non-feature columns
    y_seq = group['Class'].iloc[0]  # Single target per sequence
    X_sequences.append(X_seq)
    y_sequences.append(y_seq)

# Convert to numpy arrays
X = np.array(X_sequences)  # Shape: (samples, timesteps, features)
y = np.array(y_sequences)  # Shape: (samples,)

# One-hot encode y
y = encoder.fit_transform(y.reshape(-1, 1))  # Reshape for encoder input

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check shapes
print("X_train shape:", X_train.shape)  # (samples, timesteps, features)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)  # (samples, num_classes)
print("y_test shape:", y_test.shape)

# Model-ready data


X_train shape: (32, 76, 18)
X_test shape: (9, 76, 18)
y_train shape: (32, 2)
y_test shape: (9, 2)


In [24]:
print(y_train)

[[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [25]:
import os

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [26]:
import numpy as np
actions = np.unique(y)  # Extract unique classes from the target variable

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(76,18)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [27]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [28]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
1/1 [==============================] - 4s 4s/step - loss: 828.8743 - categorical_accuracy: 0.5625
Epoch 2/2000
1/1 [==============================] - 0s 93ms/step - loss: 950.7822 - categorical_accuracy: 0.4688
Epoch 3/2000
1/1 [==============================] - 0s 80ms/step - loss: 652.8853 - categorical_accuracy: 0.5000
Epoch 4/2000
1/1 [==============================] - 0s 84ms/step - loss: 604.0814 - categorical_accuracy: 0.5312
Epoch 5/2000
1/1 [==============================] - 0s 82ms/step - loss: 444.9615 - categorical_accuracy: 0.5000
Epoch 6/2000
1/1 [==============================] - 0s 82ms/step - loss: 526.4617 - categorical_accuracy: 0.5312
Epoch 7/2000
1/1 [==============================] - 0s 84ms/step - loss: 71.1435 - categorical_accuracy: 0.6562
Epoch 8/2000
1/1 [==============================] - 0s 79ms/step - loss: 213.8181 - categorical_accuracy: 0.4688
Epoch 9/2000
1/1 [==============================] - 0s 84ms/step - loss: 322.0980 - categorical_acc

In [30]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 76, 64)            21248     
                                                                 
 lstm_10 (LSTM)              (None, 76, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dense_9 (Dense)             (None, 2)                 66        
                                                                 
Total params: 175,778
Trainable params: 175,778
Non-tr

In [32]:
res = model.predict(X_test)
np.argmax(res[4])

1/1 [==============================] - 0s 36ms/step


1

In [37]:
model.save('LSTM_23-01-25.h5')

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import math
from tensorflow.keras.models import load_model

# Load the trained LSTM model (ensure you've saved your model after training)
model = load_model('LSTM_23-01-25.h5')  # Replace with your model path

# Initialize Mediapipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Initialize webcam
cap = cv2.VideoCapture(1)  # Webcam ID 1

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    radians = math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0])
    angle = abs(math.degrees(radians))
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Define the angle mapping for the frontal plane
frontal_plane_angles = [
    ("left_elbow", 11, 13, 15),
    ("right_elbow", 12, 14, 16),
    ("left_shoulder", 23, 11, 13),
    ("right_shoulder", 24, 12, 14),
    ("left_hip", 25, 23, 11),
    ("right_hip", 26, 24, 12),
    ("left_knee", 23, 25, 27),
    ("right_knee", 24, 26, 28),
]

# Function to calculate distances between selected landmarks
def calculate_distances(landmarks):
    relevant_pairs = [
        (11, 13),  # Left shoulder -> Left elbow
        (12, 14),  # Right shoulder -> Right elbow
        (23, 11),  # Left hip -> Left shoulder
        (24, 12),  # Right hip -> Right shoulder
        (25, 23),  # Left hip -> Left knee
        (26, 24),  # Right hip -> Right knee
        (27, 25),  # Left knee -> Left ankle
        (28, 26),  # Right knee -> Right ankle
        (11, 12),  # Left shoulder -> Right shoulder
        (13, 14),  # Left elbow -> Right elbow
    ]

    distances = []
    for i, j in relevant_pairs:
        dx = landmarks[j].x - landmarks[i].x
        dy = landmarks[j].y - landmarks[i].y
        distance = math.sqrt(dx ** 2 + dy ** 2)
        distances.append(distance)
    
    return distances

# Prepare for sequence collection
sequence_length = 20  # Use 20 frames for each sequence
sequence = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the frame
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks and calculate distances and angles
        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                # Calculate distances
                distances = calculate_distances(landmarks)

                # Calculate angles in the frontal plane
                angles = []
                for name, idx_a, idx_b, idx_c in frontal_plane_angles:
                    a = [landmarks[idx_a].x, landmarks[idx_a].y]
                    b = [landmarks[idx_b].x, landmarks[idx_b].y]
                    c = [landmarks[idx_c].x, landmarks[idx_c].y]
                    angle = calculate_angle(a, b, c)
                    angles.append(angle)

                # Combine distances and angles as features for prediction
                features = distances + angles
                sequence.append(features)

                # Ensure the sequence is of the correct length
                if len(sequence) > sequence_length:
                    sequence.pop(0)  # Remove the first frame to keep the sequence length fixed

                # If the sequence is full, predict the class
                if len(sequence) == sequence_length:
                    # Convert the sequence to a NumPy array and reshape for LSTM input
                    sequence_data = np.array(sequence).reshape(1, sequence_length, len(features))

                    # Make prediction
                    predicted_class = model.predict(sequence_data)
                    predicted_class = np.argmax(predicted_class, axis=1)  # Assuming one-hot encoding output

                    # Display the predicted class on the frame
                    cv2.putText(
                        image,
                        f'Predicted Class: {predicted_class[0]}',
                        (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),
                        2,
                        cv2.LINE_AA
                    )

        except Exception as e:
            print(f"Error processing frame: {e}")

        # Render landmarks
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

        # Show the frame with the predicted class
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


AttributeError: module 'google.protobuf.message_factory' has no attribute 'GetMessageClass'